## Common parameters for the AES\-like cipher



In [1]:
# We initialize the Galois field with the given prime p.
# This field will be used to construct matrices and vectors, so that their
# elements live in this field.
p = 11
field = GF(p)

# We initialize the block size of the cipher, which will always be the same
# for all kinds of instantiations.
blocksize = 8

# We initialize the matrix that will be used for the linear transformation
# function.
linear_transform_matrix = Matrix(field, [
    [2, 5],
    [1, 7],
])

## AES\-like cipher implementation \(Tasks 1 & 2\)



In [2]:
# Subkey generation generic wrapper.
def gen_subkey(master_key, round_num, gen_subkey_func):
    return gen_subkey_func(master_key, round_num)

# Subkey addition function.
def add_subkey(state, subkey, inverse=False):
    # We construct the round key by concatenating the given subkey with a
    # copy of itself. Then, we add (or subtract) the round key to the state
    # vector element by element.
    round_key = vector(list(subkey) + list(subkey))
    for i in range(len(state)):
        if not inverse:
            state[i] += round_key[i]
        else:
            state[i] -= round_key[i]
    return state

# Substitution function generic wrapper.
def substitution(state, sub_func, inverse=False):
    return sub_func(state, inverse)

# Transposition function.
# Note: this function is the inverse of itself.
def transposition(state):
    # We leave the first half of the state vector unchanged and reverse the
    # second half. Finally, we stick the two halves back toghether.
    first_half = list(state[0:blocksize/2])
    second_half = list(reversed(state[blocksize/2:blocksize]))
    return vector(first_half + second_half)

# Linear transformation function.
def linear(state, inverse=False):
    # We transform the state vector by rows into a matrix, then we perform
    # matrix-matrix multiplication with the linear transformation matrix.
    # Finally, Transform the state matrix back into a vector, again by rows.
    state_matrix = Matrix([state[0:4], state[4:8]])
    if not inverse:
        transform_matrix = linear_transform_matrix
    else:
        transform_matrix = linear_transform_matrix.inverse()
    state_matrix = transform_matrix * state_matrix
    return vector(list(state_matrix[0]) + list(state_matrix[1]))

# Encryption function wrapper.
def enc_aeslike(key, plaintext, gen_subkey_func, sub_func):
    # First, we set plaintext as initial state vector.
    state = copy(plaintext)
    # We execute the first 4 rounds, applying for each round the following
    # operations: KeyAddition, Substitution, Transposition, LinearTransformation.
    for i in range(4):
        ki = gen_subkey(key, i, gen_subkey_func)
        state = add_subkey(state, ki)
        state = substitution(state, sub_func)
        state = transposition(state)
        state = linear(state)
    # Then, we execute the 5th and final round, which consists of the following
    # operations: KeyAddition, Substitution, Transposition.
    k5 = gen_subkey(key, 4, gen_subkey_func)
    state = add_subkey(state, k5)
    state = substitution(state, sub_func)
    state = transposition(state)
    # Finally, we perform a 6th KeyAddition to "seal" the ciphertext.
    k6 = gen_subkey(key, 5, gen_subkey_func)
    return add_subkey(state, k6)

# Decryption function wrapper.
def dec_aeslike(key, ciphertext, gen_subkey_func, sub_func):
    # First, we set ciphertext as initial state vector.
    state = copy(ciphertext)
    # We invert the last KeyAddition that "seals" the ciphertext.
    k6 = gen_subkey(key, 5, gen_subkey_func)
    state = add_subkey(state, k6, inverse=True)
    # Then, we invert the 5th round, by performing the following operations:
    # Transposition, Substitution, KeyAddition.
    state = transposition(state)
    state = substitution(state, sub_func, inverse=True)
    k5 = gen_subkey(key, 4, gen_subkey_func)
    state = add_subkey(state, k5, inverse=True)
    # Finally, we invert the remaining rounds, by performing the following operations:
    # LinearTransformation, Transposition, Substitution, KeyAddition.
    for i in range(4):
        state = linear(state, inverse=True)
        state = transposition(state)
        state = substitution(state, sub_func, inverse=True)
        ki = gen_subkey(key, 3-i, gen_subkey_func) # Reverse round number.
        state = add_subkey(state, ki, inverse=True)
    return state

## Linear instantiation of the AES\-like cipher



In [3]:
# For the linear instantiation, the key length is the same as the cipher
# block size.
keylength_linear = blocksize

# Indexes of the master key for the subkey generation.
# Note: these are 1-indexed, while all the computation are 0-indexed.
subkey_indexes_linear = [
    [1, 3, 5, 7],
    [1, 2, 3, 4],
    [1, 4, 5, 8],
    [1, 4, 6, 7],
    [1, 3, 6, 8],
    [3, 4, 5, 6],
]

# Subkey generation function.
def gen_subkey_linear(master_key, round_num):
    # We construct the subkey for the given round number with the entries of
    # master key, using the indexes at the corresponding row in subkey_indexes_linear.
    subkey = []
    for index in subkey_indexes_linear[round_num]:
        subkey.append(master_key[index-1])
    return vector(subkey)

# Substitution function.
def substitution_linear(state, inverse=False):
    # We multiply (or divide) every element of the state vector by 2.
    for i in range(len(state)):
        if not inverse:
            state[i] *= field(2)
        else:
            state[i] /= field(2)
    return state

# Encryption function.
def enc_aeslike_linear(key, plaintext):
    return enc_aeslike(key, plaintext, gen_subkey_linear, substitution_linear)

# Decryption function.
def dec_aeslike_linear(key, ciphertext):
    return dec_aeslike(key, ciphertext, gen_subkey_linear, substitution_linear)

#### Linear instantiation implementation correctness check



In [4]:
# We initialize the test vectors (for the only test we have available).
plaintext = vector(field, [1, 0, 0, 0, 0, 0, 0, 0])
expected_ciphertext = vector(field, [4, 0, 0, 9, 7, 0, 0, 3])
key = vector(field, [1, 0, 0, 0, 0, 0, 0, 0])

# We execute the encryption and check if the result matches.
ciphertext = enc_aeslike_linear(key, plaintext)

if ciphertext == expected_ciphertext:
    print('Linear instantiation, encryption implementation correctness check: PASS')
else:
    print('Linear instantiation, encryption implementation correctness check: FAIL')

# We execute the decryption and check if we get back the original plaintext.
recovered_plaintext = dec_aeslike_linear(key, ciphertext)
if recovered_plaintext == plaintext:
    print('Linear instantiation, decryption implementation correctness check: PASS')
else:
    print('Linear instantiation, decryption implementation correctness check: FAIL')

Linear instantiation, encryption implementation correctness check: PASS
Linear instantiation, decryption implementation correctness check: PASS


#### Breaking the linear instantiation \(Task 3\)



In [5]:
# The linear instantiation of the cipher can be reduced to the following linear
# function.
#
#     ciphertext = MatrixA * key + MatrixB * plaintext
#
# Therefore, we want to find matrices A and B.

# We find matrix A by setting the plaintext to the zero vector and the key
# progressively to the next standard orthonormal basis vector. Basically, we
# exploit the following expression.
#
#     ciphertext = MatrixA * key + MatrixB * 0
#                = MatrixA * key
#
A_linear = Matrix(field, keylength_linear, keylength_linear)
plaintext = vector(field, [0, 0, 0, 0, 0, 0, 0, 0])
for i in range(keylength_linear):
    key = vector(field, [0, 0, 0, 0, 0, 0, 0, 0])
    key[i] = 1
    A_linear.set_column(i, enc_aeslike_linear(key, plaintext))

# Similarly, we find matrix B. In this case the roles of key and plaintext are
# switched.
B_linear = Matrix(field, blocksize, blocksize)
key = vector(field, [0, 0, 0, 0, 0, 0, 0, 0])
for i in range(blocksize):
    plaintext = vector(field, [0, 0, 0, 0, 0, 0, 0, 0])
    plaintext[i] = 1
    B_linear.set_column(i, enc_aeslike_linear(key, plaintext))

print('''The resulting matrices, which should provide the linear relationship in the
linear instantiation of the AES-like cipher, are:''')
print('Matrix A:')
print(A_linear)
print('Matrix B:')
print(B_linear)

The resulting matrices, which should provide the linear relationship in the
linear instantiation of the AES-like cipher, are:
Matrix A:
[ 9  0  1  6  0  0  1 10]
[ 0  8  6  2  2  9  0  0]
[ 0  6  0  8  3 10  0  0]
[ 6  0  0  8  0  1  6  6]
[ 2  0  1 10  0  0  1  3]
[ 0  1  8  4  9  6  0  0]
[ 0 10  0  5  7  6  0  0]
[ 3  0  0  1  0  1  4  8]
Matrix B:
[6 0 0 3 3 0 0 0]
[0 6 3 0 0 3 0 0]
[0 3 6 0 0 0 3 0]
[3 0 0 6 0 0 0 3]
[5 0 0 0 4 0 0 8]
[0 5 0 0 0 4 8 0]
[0 0 5 0 0 8 4 0]
[0 0 0 5 8 0 0 4]


In [6]:
# We initialize the test vectors (for the only test we have available).
plaintext = vector(field, [1, 0, 0, 0, 0, 0, 0, 0])
expected_ciphertext = vector(field, [4, 0, 0, 9, 7, 0, 0, 3])
key = vector(field, [1, 0, 0, 0, 0, 0, 0, 0])

# We check the correctness of the two matrices by verifying if the overall linear
# relationship correctly encrypts the test vector plaintext.
ciphertext = A_linear * key + B_linear * plaintext
if ciphertext == expected_ciphertext:
    print('Breaking the linear instantiation, correctness check: PASS')
else:
    print('Breaking the linear instantiation, correctness check: FAIL')

# We verify that we can successfully recover the correct key exploiting the linear
# relationship, using the two matrices.
recovered_key = A_linear.inverse() * (ciphertext - B_linear * plaintext)
if recovered_key == key:
    print('Breaking the linear instantiation, key recovery check: PASS')
else:
    print('Breaking the linear instantiation, key recovery check: FAIL')

Breaking the linear instantiation, correctness check: PASS
Breaking the linear instantiation, key recovery check: PASS


#### Known-plaintext attack to the linear instantiation \(Task 4\)



In [7]:
import ast

# For each KPA sample provided in the KPApairsZUC_linear.txt file, we recover
# the key using the matrices A and B we previously found.
recovered_keys = []
with open('KPApairsZUC_linear.txt') as KPA_data:
    for KPA_sample in KPA_data:
        plaintext_raw, ciphertext_raw = KPA_sample.rstrip().split('\t')
        plaintext = vector(field, ast.literal_eval(plaintext_raw))
        ciphertext = vector(field, ast.literal_eval(ciphertext_raw))
        recovered_keys.append(
            A_linear.inverse() * (ciphertext - B_linear * plaintext)
        )

print('''The keys recovered using a known-plaintext attack on the linear
instantiation of the AES-like cipher are:''')
for i, key in enumerate(recovered_keys):
    print(f'Key from KPA sample #{i+1}: {key}')

# Since we know that all the given samples should be encrypted with the same key,
# we check the we indeed recovered the same key from all KPA samples.
if all(key == recovered_keys[0] for key in recovered_keys):
    print('Breaking the linear instantiation, known-plaintext attack: PASS')
else:
    print('Breaking the linear instantiation, known-plaintext attack: FAIL')

The keys recovered using a known-plaintext attack on the linear
instantiation of the AES-like cipher are:
Key from KPA sample #1: (3, 2, 6, 5, 8, 9, 8, 10)
Key from KPA sample #2: (3, 2, 6, 5, 8, 9, 8, 10)
Key from KPA sample #3: (3, 2, 6, 5, 8, 9, 8, 10)
Key from KPA sample #4: (3, 2, 6, 5, 8, 9, 8, 10)
Key from KPA sample #5: (3, 2, 6, 5, 8, 9, 8, 10)
Breaking the linear instantiation, known-plaintext attack: PASS


## Nearly linear instantiation of the AES\-like cipher (Task 5)



In [8]:
# For the nearly linear instantiation, the key length is the same as the
# cipher block size.
keylength_nearly_linear = blocksize

# In the nearly linear instantiation we use the following table to perform
# the substitution function (and its inverse).
substitution_table = {0:0, 1:2, 2:4, 3:8, 4:6, 5:10, 6:1, 7:3, 8:5, 9:7, 10:9}
inverse_substitution_table = {0:0, 1:6, 2:1, 3:7, 4:2, 5:8, 6:4, 7:9, 8:3, 9:10, 10:5}

# Subkey generation function.
# Note: the nearly linear instantiation uses the same function as the
# linear instantiation.
def gen_subkey_nearly_linear(master_key, round_num):
    return gen_subkey_linear(master_key, round_num)

# Substitution function.
def substitution_nearly_linear(state, inverse=False):
    for i in range(len(state)):
        if not inverse:
            state[i] = substitution_table[state[i]]
        else:
            state[i] = inverse_substitution_table[state[i]]
    return state

# Encryption function.
def enc_aeslike_nearly_linear(key, plaintext):
    return enc_aeslike(key, plaintext, gen_subkey_linear, substitution_nearly_linear)

# Decryption function.
def dec_aeslike_nearly_linear(key, ciphertext):
    return dec_aeslike(key, ciphertext, gen_subkey_linear, substitution_nearly_linear)

#### Nearly linear instantiation implementation correctness check



In [9]:
# We initialize the test vectors (for the only test we have available).
plaintext = vector(field, [1, 0, 0, 0, 0, 0, 0, 0])
expected_ciphertext = vector(field, [9, 0, 0, 0, 5, 0, 0, 6])
key = vector(field, [1, 0, 0, 0, 0, 0, 0, 0])

# We execute the encryption and check if the result matches.
ciphertext = enc_aeslike_nearly_linear(key, plaintext)
if ciphertext == expected_ciphertext:
    print('Nearly linear instantiation, encryption implementation correctness check: PASS')
else:
    print('Nearly linear instantiation, encryption implementation correctness check: FAIL')

# We execute the decryption and check if we get back the original plaintext.
recovered_plaintext = dec_aeslike_nearly_linear(key, ciphertext)
if recovered_plaintext == plaintext:
    print('Nearly linear instantiation, decryption implementation correctness check: PASS')
else:
    print('Nearly linear instantiation, decryption implementation correctness check: FAIL')

Nearly linear instantiation, encryption implementation correctness check: PASS
Nearly linear instantiation, decryption implementation correctness check: PASS


#### Breaking the nearly linear instantiation \(Task 6\)



In [13]:
# Since for 9 values out 11 the substitution table of nearly linear
# instantiation of the cipher gives the same values as the substitution
# function of the linear instantiation, we assume that the linear
# instantiation is a good linear approximation of the nearly linear
# instantiation. (<- This sentence is a bit wordy. A better explanation
# is given in the report.)
A_nearly_linear, B_nearly_linear = copy(A_linear), copy(B_linear)

import time
def evaluate_linear_relation_approximation(A, B, enc_func, runs):
    # We generate many random plaintexts and corresponding encryption keys.
    start_time = time.time()
    approximation_test_runs = runs
    approximation_test_plaintexts = [random_vector(field, blocksize) for _ in range(approximation_test_runs)]
    approximation_test_keys = [random_vector(field, keylength_nearly_linear) for _ in range(approximation_test_runs)]
    end_time = time.time()
    print(f'Generated {approximation_test_runs} plaintexts and keys to test the approximation (took {end_time - start_time:.2f} s)')
    # We encrypt each plaintext with its corresponding encryption key, both
    # using the approximating matrices and using the true encryption function.
    # When both results in the same ciphertext, we increment the number of
    # successful approximations.
    start_time = time.time()
    approximation_test_successes = 0
    for i in range(approximation_test_runs):
        approximated_ciphertext = A * approximation_test_keys[i] + B * approximation_test_plaintexts[i]
        expected_ciphertext = enc_func(approximation_test_keys[i], approximation_test_plaintexts[i])
        if approximated_ciphertext == expected_ciphertext:
            approximation_test_successes += 1
    end_time = time.time()
    approximation_test_prob = float(approximation_test_successes / approximation_test_runs)
    print(f'Approximation success probability: {100 * approximation_test_prob:.4f} % (took {end_time - start_time:.2f} s)')
    # We compute the ratio between the estimated success probability of
    # the approximation and the probability of correctly guessing a random
    # ciphertext.
    if approximation_test_prob > 0:
        approximation_test_ratio = float(approximation_test_prob / (1 / pow(p, blocksize)))
        print(f'Ratio of approximation success probability wrt randomly picking a ciphertext: {approximation_test_ratio:.2g}')
    return approximation_test_prob

# Note: this computation requires a lot of time, so higher values of the
# `runs` parameter may be tested on a local machine or with a non-free tier
# CoCalc subscription.
_ = evaluate_linear_relation_approximation(
    A_nearly_linear, B_nearly_linear,
    enc_aeslike_nearly_linear,
    50_000
)

Generated 50000 plaintexts and keys to test the approximation (took 7.67 s)


Approximation success probability: 0.0380 % (took 151.18 s)
Ratio of approximation success probability wrt randomly picking a ciphertext: 8.1e+04


## Non\-linear instantiation of the AES\-like cipher \(Task 7\)



In [12]:
# For the non-linear instantiation, the key length is half the cipher
# block size.
keylength_nonlinear = int(blocksize/2)

# Indexes of the master key for the subkey generation.
# Note: these are 1-indexed, while all the computation are 0-indexed.
subkey_indexes_nonlinear = [
    [1, 2, 3, 4],
    [1, 2, 4, 3],
    [2, 3, 4, 1],
    [1, 4, 2, 3],
    [3, 4, 1, 2],
    [2, 4, 1, 3],
]

# Subkey generation function.
def gen_subkey_nonlinear(master_key, round_num):
    # We construct the subkey for the given round number with the entries of master
    # key, using the indexes at the corresponding row in subkey_indexes_nonlinear.
    subkey = []
    for index in subkey_indexes_nonlinear[round_num]:
        subkey.append(master_key[index-1])
    return vector(subkey)

# Substitution function.
def substitution_nonlinear(state, inverse=False):
    for i in range(len(state)):
        # If the current element is zero, we define its inverse to be zero, hence
        # no modification is necessary.
        if state[i] == 0:
            continue
        # Otherwise, first (or last) we take the inverse of the current element of
        # the state vector, and then (or first) we multiply (or divide) it by 2.
        if not inverse:
            state[i] = 2 * state[i].inverse()
        else:
            state[i] = (state[i] / 2).inverse()
    return state

# Encryption function.
def enc_aeslike_nonlinear(key, plaintext):
    return enc_aeslike(key, plaintext, gen_subkey_nonlinear, substitution_nonlinear)

# Decryption function.
def dec_aeslike_nonlinear(key, ciphertext):
    return dec_aeslike(key, ciphertext, gen_subkey_nonlinear, substitution_nonlinear)

#### Non-linear instantiation implementation correctness check



In [14]:
# We initialize the test vectors (for the only test we have available).
plaintext = vector(field, [1, 0, 0, 0, 0, 0, 0, 0])
expected_ciphertext = vector(field, [5, 0, 3, 2, 5, 2, 1, 1])
key = vector(field, [1, 0, 0, 0])

# We execute the encryption and check if the result matches.
ciphertext = enc_aeslike_nonlinear(key, plaintext)
if ciphertext == expected_ciphertext:
    print('Non-linear instantiation, encryption implementation correctness check: PASS')
else:
    print('Non-linear instantiation, encryption implementation correctness check: FAIL')

# We execute the decryption and check if we get back the original plaintext.
recovered_plaintext = dec_aeslike_nonlinear(key, ciphertext)
if recovered_plaintext == plaintext:
    print('Non-linear instantiation, decryption implementation correctness check: PASS')
else:
    print('Non-linear instantiation, decryption implementation correctness check: FAIL')

Non-linear instantiation, encryption implementation correctness check: PASS
Non-linear instantiation, decryption implementation correctness check: PASS


#### Breaking the non-linear instantiation via meet-in-the-middle attack \(Task 8\)



In [32]:
# We initialize a random test vector to verify the correctness of our
# implementation of the meet-in-the-middle procedure.
mitm_origin_plaintext = random_vector(field, blocksize)
mitm_origin_k1 = random_vector(field, keylength_nonlinear)
mitm_origin_k2 = random_vector(field, keylength_nonlinear)
mitm_origin_ciphertext = enc_aeslike_nonlinear(
    mitm_origin_k2, enc_aeslike_nonlinear(mitm_origin_k1, mitm_origin_plaintext)
)

print(f'Meet-in-the-middle implementation test, original plaintext: {mitm_origin_plaintext}')
print(f'Meet-in-the-middle implementation test, original key: ({mitm_origin_k1}, {mitm_origin_k2})')
print(f'Meet-in-the-middle implementation test, original ciphertext: {mitm_origin_ciphertext}')

# We generate all possible encryption and decryption keys.
# Note: the total number of keys is 11^4 + 11^4.
import itertools
mitm_enc_keys = [vector(field, v) for v in itertools.product(range(p), repeat=keylength_nonlinear)]
mitm_dec_keys = [vector(field, v) for v in itertools.product(range(p), repeat=keylength_nonlinear)]

# We encrypt the plaintext test vector we all possible encryption keys.
mitm_ciphertexts = dict()
for i, enc_key in enumerate(mitm_enc_keys):
    mitm_ciphertext = enc_aeslike_nonlinear(enc_key, mitm_origin_plaintext)
    mitm_ciphertext.set_immutable()
    mitm_ciphertexts[mitm_ciphertext] = i

# We decrypt the ciphertext test vector we all possible decryption keys,
# then we check if the result of the decryption is equal to one of the
# encryptions of the plaintext we performed above. If so, we add the
# corresponding keys to the set of candidate keys.
mitm_candidate_keys = []
for dec_key in mitm_dec_keys:
    mitm_plaintext = dec_aeslike_nonlinear(dec_key, mitm_origin_ciphertext)
    mitm_plaintext.set_immutable()
    if mitm_plaintext in mitm_ciphertexts:
        mitm_candidate_keys.append(
            (mitm_enc_keys[mitm_ciphertexts[mitm_plaintext]], dec_key)
        )

# We loop through all the candidate keys we've found and we check whether
# they are correct or not.
# The implementation test is successful if one, and only one, candidate key
# results correct.
print(f'Meet-in-the-middle implementation test, found {len(mitm_candidate_keys)} candidate keys')
for i, candidate_key in enumerate(mitm_candidate_keys):
    print(f'Meet-in-the-middle implementation test, candidate key #{i+1}: ({candidate_key[0]}, {candidate_key[1]})')
    if candidate_key[0] == mitm_origin_k1 and candidate_key[1] == mitm_origin_k2:
        print(f'Meet-in-the-middle implementation test, candidate key #{i+1} is CORRECT')
    else:
        print(f'Meet-in-the-middle implementation test, candidate key #{i+1} is INCORRECT')

Meet-in-the-middle implementation test, original plaintext: (2, 6, 10, 3, 10, 2, 10, 0)
Meet-in-the-middle implementation test, original key: ((3, 10, 0, 2), (8, 2, 2, 8))
Meet-in-the-middle implementation test, original ciphertext: (8, 9, 1, 0, 10, 9, 3, 10)


Meet-in-the-middle implementation test, found 2 candidate keys
Meet-in-the-middle implementation test, candidate key #1: ((1, 6, 8, 4), (2, 10, 8, 1))
Meet-in-the-middle implementation test, candidate key #1 is INCORRECT
Meet-in-the-middle implementation test, candidate key #2: ((3, 10, 0, 2), (8, 2, 2, 8))
Meet-in-the-middle implementation test, candidate key #2 is CORRECT


In [35]:
# Here, we repeat the same procedure as above, but this time we use the
# five given plaintext-ciphertext pairs.

import ast
import itertools

# TODO(simone): comment
mitm_origin_plaintexts = []
mitm_origin_ciphertexts = []
with open('KPApairsZUC_non_linear.txt') as KPA_data:
    for KPA_sample in KPA_data:
        plaintext_raw, ciphertext_raw = KPA_sample.rstrip().split('\t')
        plaintext = vector(field, ast.literal_eval(plaintext_raw))
        ciphertext = vector(field, ast.literal_eval(ciphertext_raw))
        mitm_origin_plaintexts.append(plaintext)
        mitm_origin_ciphertexts.append(ciphertext)

for i in range(len(mitm_origin_plaintexts)):
    print(f'Meet-in-the-middle, original plaintext #{i+1}: {mitm_origin_plaintexts[i]}')
    print(f'Meet-in-the-middle, original ciphertext #{i+1}: {mitm_origin_ciphertexts[i]}')

# We generate all possible encryption and decryption keys.
# Note: the total number of keys is 11^4 + 11^4.
mitm_enc_keys = [vector(field, v) for v in itertools.product(range(p), repeat=keylength_nonlinear)]
mitm_dec_keys = [vector(field, v) for v in itertools.product(range(p), repeat=keylength_nonlinear)]

# We encrypt the plaintext test vector we all possible encryption keys.
mitm_ciphertexts = dict()
for i, enc_key in enumerate(mitm_enc_keys):
    mitm_ciphertext = enc_aeslike_nonlinear(enc_key, mitm_origin_plaintexts[0])
    mitm_ciphertext.set_immutable()
    mitm_ciphertexts[mitm_ciphertext] = i

# We decrypt the ciphertext test vector we all possible decryption keys,
# then we check if the result of the decryption is equal to one of the
# encryptions of the plaintext we performed above. If so, we add the
# corresponding keys to the set of candidate keys.
mitm_candidate_keys = []
for dec_key in mitm_dec_keys:
    mitm_plaintext = dec_aeslike_nonlinear(dec_key, mitm_origin_ciphertexts[0])
    mitm_plaintext.set_immutable()
    if mitm_plaintext in mitm_ciphertexts:
        mitm_candidate_keys.append(
            (mitm_enc_keys[mitm_ciphertexts[mitm_plaintext]], dec_key)
        )

# We loop through all the candidate keys we've found and for each of
# these we perform two checks as explained below, if both pass the
# corresponding candidate key is the considered the correct one.
print(f'Meet-in-the-middle, found {len(mitm_candidate_keys)} candidate keys')
for i, candidate_key in enumerate(mitm_candidate_keys):
    print(f'Meet-in-the-middle, candidate key #{i+1}: ({candidate_key[0]}, {candidate_key[1]})')
    candidate_key_is_correct = True
    for j in range(len(mitm_origin_plaintexts)):
        # First check: the candidate key must correctly encrypt the
        # plaintext to the corresponding ciphertext.
        candidate_ciphertext = enc_aeslike_nonlinear(
            candidate_key[1], enc_aeslike_nonlinear(candidate_key[0], mitm_origin_plaintexts[j])
        )
        if candidate_ciphertext != mitm_origin_ciphertexts[j]:
            candidate_key_is_correct = False
            break
        # Second check: the candidate key must correctly decrypt the
        # ciphertext to the corresponding plaintext.
        candidate_plaintext = dec_aeslike_nonlinear(
            candidate_key[0], dec_aeslike_nonlinear(candidate_key[1], mitm_origin_ciphertexts[j])
        )
        if candidate_plaintext != mitm_origin_plaintexts[j]:
            candidate_key_is_correct = False
            break
    # If both the above checks pass, the current candidate key is the
    # considered correct.
    # Note: only one candidate key should turns out to be correct.
    if candidate_key_is_correct:
        print(f'Meet-in-the-middle, candidate key #{i+1} is CORRECT')
    else:
        print(f'Meet-in-the-middle, candidate key #{i+1} is INCORRECT')

Meet-in-the-middle, original plaintext #1: (1, 8, 3, 2, 2, 0, 6, 4)
Meet-in-the-middle, original ciphertext #1: (7, 3, 7, 6, 2, 9, 2, 4)
Meet-in-the-middle, original plaintext #2: (8, 5, 4, 0, 2, 5, 4, 9)
Meet-in-the-middle, original ciphertext #2: (10, 2, 3, 9, 6, 3, 9, 4)
Meet-in-the-middle, original plaintext #3: (0, 4, 5, 1, 5, 9, 2, 1)
Meet-in-the-middle, original ciphertext #3: (6, 4, 1, 0, 9, 10, 6, 1)
Meet-in-the-middle, original plaintext #4: (0, 7, 0, 3, 10, 8, 2, 7)
Meet-in-the-middle, original ciphertext #4: (2, 5, 2, 2, 6, 6, 2, 10)
Meet-in-the-middle, original plaintext #5: (1, 6, 9, 9, 3, 5, 7, 7)
Meet-in-the-middle, original ciphertext #5: (2, 6, 1, 2, 4, 6, 7, 8)


Meet-in-the-middle, found 2 candidate keys
Meet-in-the-middle, candidate key #1: ((8, 10, 1, 7), (0, 8, 5, 7))
Meet-in-the-middle, candidate key #1 is INCORRECT
Meet-in-the-middle, candidate key #2: ((10, 6, 1, 6), (2, 8, 9, 3))
Meet-in-the-middle, candidate key #2 is CORRECT
